In [41]:
import pandas as pd

companies = pd.read_csv("companies.txt", delimiter="\t", encoding='ISO-8859-1')
round2 = pd.read_csv("rounds2.csv", encoding='ISO-8859-1')

# Setting the value of key column to lower case to ensure consistency with joining dataframe
companies['permalink'] = companies['permalink'].str.lower()
round2['company_permalink'] = round2['company_permalink'].str.lower()

# Replacing all NaN (Null) values with 0 to ensure that we get consistent aggregations
round2['raised_amount_usd'].fillna(0, inplace=True)

# Table 1.1 outcome
print("\n\nTable 1.1")
print("----------\n")

# Get details of unique companies in companies file
print("Unique values in companies DataFrame :", companies['permalink'].nunique())

# Get details of unique companies in round2
print("Unique values in companies round2 :", round2['company_permalink'].nunique())

# Find out of there are companies in round2 that are not present in companies file

df1 = pd.DataFrame(companies,columns=['permalink'])
df2 = pd.DataFrame(round2,columns=['company_permalink'])

set_diff_df = pd.concat([df2, df1, df1], sort=True).drop_duplicates(keep=False)
print("Additional companies in round2 that does not exist in companies DataFrame :", set_diff_df['company_permalink'].count())

# Merger two data frames
master_frame = pd.merge(round2, companies, how='outer', right_on=['permalink'], left_on=['company_permalink'])
observations = master_frame['company_permalink'].count()

print ("Number of observations in master frame", observations)




Table 1.1
----------

Unique values in companies DataFrame : 66368
Unique values in companies round2 : 66370
Additional companies in round2 that does not exist in companies DataFrame : 42493
Number of observations in master frame 114949


Table 2.1
----------

Investment amount by funding type as 'VENTURE'        =  10634054
Investment amount by funding type as 'ANGEL'          =  764564
Investment amount by funding type as 'SEED'           =  556607
Investment amount by funding type as 'PRIVATE EQUITY' =  62111788


Table 3.1
----------

country_code
USA   422510842796
CHN    39835418773
GBR    20245627416
IND    14391858718
CAN     9583332317
FRA     7259536732
ISR     6907514579
DEU     6346959822
JPN     3363676611
Name: raised_amount_usd, dtype: float64


Table 4.1
----------



Table 5.1
----------



Funding type = 'venture' and Country='USA'


Total number of investments (count) :  35759
Total amount of investment (USD) :  422510842796
                                        

In [45]:
# Table 2.1 outcome
print("\n\nTable 2.1")
print("----------\n")

# Setting display format for float values that are an outcome of groupby
pd.set_option('display.float_format', lambda x: '%.0f' %x)

# Calculate the most representative value of investment amount by funding type = 'VENTURE'
venture_usd = round2[round2['funding_round_type'] == 'venture'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'VENTURE'        = ", '%.0f' % venture_usd)

# Calculate the most representative value of investment amount by funding type = 'ANGEL'
angel_usd = round2[round2['funding_round_type'] == 'angel'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'ANGEL'          = ", '%.0f' % angel_usd)

# Calculate the most representative value of investment amount by funding type = 'SEED'
seed_usd = round2[round2['funding_round_type'] == 'seed'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'SEED'           = ", '%.0f' % seed_usd)

# Calculate the most representative value of investment amount by funding type = 'PRIVATE EQUITY'
private_equity_usd = round2[round2['funding_round_type'] == 'private_equity'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'PRIVATE EQUITY' = ", '%.0f' % private_equity_usd)



Table 2.1
----------

Investment amount by funding type as 'VENTURE'        =  10634054
Investment amount by funding type as 'ANGEL'          =  764564
Investment amount by funding type as 'SEED'           =  556607
Investment amount by funding type as 'PRIVATE EQUITY' =  62111788


In [44]:
# Table 3.1 outcome
print("\n\nTable 3.1")
print("----------\n")

#For the chosen investment type, make a data frame named top9 with the top nine countries 
top9 = master_frame[master_frame['funding_round_type'] == 'venture'].groupby('country_code').sum()
print(top9['raised_amount_usd'].nlargest(9))



Table 3.1
----------

country_code
USA   422510842796
CHN    39835418773
GBR    20245627416
IND    14391858718
CAN     9583332317
FRA     7259536732
ISR     6907514579
DEU     6346959822
JPN     3363676611
Name: raised_amount_usd, dtype: float64


In [43]:
# Table 4.1 outcome
print("\n\nTable 4.1")
print("----------\n")

# Splitting category list to extract primary sector into a new column 
new_column = master_frame['category_list'].str.split('|', n=1, expand=True)
master_frame['primary_sector'] = new_column[0]

# Load mapping file
mappings = pd.read_csv("mapping.csv", delimiter=",", encoding='ISO-8859-1')

master_frame['primary_sector'].fillna(0, inplace=True)
mappings['category_list'].fillna(0, inplace=True)

# Map each primary sector mapped to its main sector 
master_frame_mapping = pd.merge(master_frame, mappings, how='left', left_on=['primary_sector'], right_on=['category_list'])




Table 4.1
----------



In [42]:
print("\n\nTable 5.1")
print("----------\n")
print("\n\nFunding type = 'venture' and Country='USA'\n\n")

# Filter data for Country1 and Investment Type between 5 and 15 Mil
D1 = master_frame_mapping[(master_frame_mapping['funding_round_type'] == 'venture') & \
                          (master_frame_mapping['country_code'] == 'USA')]

# total number (or count) of investments for each main sector in a separate column
d1group = (D1.groupby('primary_sector')
        .agg({'raised_amount_usd':'sum',
              'Automotive & Sports':'sum',
              'Blanks':'sum',
              'Cleantech / Semiconductors':'sum',
              'Entertainment':'sum',
              'Health':'sum',
              'Manufacturing':'sum',
              'News, Search and Messaging':'sum', 
              'Others':'sum',
              'Social, Finance, Analytics, Advertising':'sum'})
        .reset_index()
        .rename(columns={'Automotive & Sports':'Automotive & Sports Count',
                         'Blanks':'Blanks Count',
                         'Cleantech / Semiconductors':'Cleantech / Semiconductors Count',
                         'Entertainment':'Entertainment Count',
                         'Health':'Health Count',
                         'Manufacturing':'Manufacturing Count',
                         'News, Search and Messaging':'News, Search and Messaging Count', 
                         'Others':'Others Count',
                         'Social, Finance, Analytics, Advertising':'Social, Finance, Analytics, Advertising Count'}))

d1_total_investment_count = d1group['Automotive & Sports Count'].sum() + \
                            d1group['Blanks Count'].sum() + \
                            d1group['Cleantech / Semiconductors Count'].sum() + \
                            d1group['Entertainment Count'].sum() + \
                            d1group['Health Count'].sum() + \
                            d1group['Manufacturing Count'].sum() + \
                            d1group['News, Search and Messaging Count'].sum() + \
                            d1group['Others Count'].sum() + \
                            d1group['Social, Finance, Analytics, Advertising Count'].sum()

print("Total number of investments (count) : ", '%.0f' % d1_total_investment_count)

#Total amount invested in each main sector in a separate column
d1_total_investment_usd = d1group.raised_amount_usd.sum()
print("Total amount of investment (USD) : ", '%.0f' % d1_total_investment_usd)

# Top sectors (based on count of investments)
df_investment1 = pd.DataFrame({'Automotive & Sports Count':d1group['Automotive & Sports Count'].sum(),\
                                'Blanks Count':d1group['Blanks Count'].sum(),\
                                'Cleantech / Semiconductors Count':d1group['Cleantech / Semiconductors Count'].sum(),\
                                'Entertainment Count':d1group['Entertainment Count'].sum(),\
                                'Health Count':d1group['Health Count'].sum(),\
                                'Manufacturing Count':d1group['Manufacturing Count'].sum(),\
                                'News, Search and Messaging Count':d1group['News, Search and Messaging Count'].sum(),\
                                'Others Count':d1group['Others Count'].sum(),\
                                'Social, Finance, Analytics, Advertising Count':d1group['Social, Finance, Analytics, Advertising Count'].sum()},index=[0])

print(pd.DataFrame(df_investment1.max().sort_values(ascending=False)))

# For the top sector count-wise (point 3), which company received the highest investment
d1company1 = D1[D1['Others'] == 1]
c1 = d1company1.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(c1)

# For the second-best sector count-wise (point 4), which company received the highest investment
d1company2 = D1[D1['Cleantech / Semiconductors'] == 1]
c2 = d1company2.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(c2)


print("\n\nFunding type = 'venture' and Country='GBR'\n\n")

# Filter data for Country1 and Investment Type between 5 and 15 Mil
D2 = master_frame_mapping[(master_frame_mapping['funding_round_type'] == 'venture') & \
                          (master_frame_mapping['country_code'] == 'GBR')]

# total number (or count) of investments for each main sector in a separate column
d2group = (D2.groupby('primary_sector')
        .agg({'raised_amount_usd':'sum',
              'Automotive & Sports':'sum',
              'Blanks':'sum',
              'Cleantech / Semiconductors':'sum',
              'Entertainment':'sum',
              'Health':'sum',
              'Manufacturing':'sum',
              'News, Search and Messaging':'sum', 
              'Others':'sum',
              'Social, Finance, Analytics, Advertising':'sum'})
        .reset_index()
        .rename(columns={'Automotive & Sports':'Automotive & Sports Count',
                         'Blanks':'Blanks Count',
                         'Cleantech / Semiconductors':'Cleantech / Semiconductors Count',
                         'Entertainment':'Entertainment Count',
                         'Health':'Health Count',
                         'Manufacturing':'Manufacturing Count',
                         'News, Search and Messaging':'News, Search and Messaging Count', 
                         'Others':'Others Count',
                         'Social, Finance, Analytics, Advertising':'Social, Finance, Analytics, Advertising Count'}))

d2_total_investment_count = d2group['Automotive & Sports Count'].sum() + \
                            d2group['Blanks Count'].sum() + \
                            d2group['Cleantech / Semiconductors Count'].sum() + \
                            d2group['Entertainment Count'].sum() + \
                            d2group['Health Count'].sum() + \
                            d2group['Manufacturing Count'].sum() + \
                            d2group['News, Search and Messaging Count'].sum() + \
                            d2group['Others Count'].sum() + \
                            d2group['Social, Finance, Analytics, Advertising Count'].sum()

print("Total number of investments (count) : ", '%.0f' % d2_total_investment_count)

#Total amount invested in each main sector in a separate column
d2_total_investment_usd = d2group.raised_amount_usd.sum()
print("Total amount of investment (USD) : ", '%.0f' % d2_total_investment_usd)

# Top sectors (based on count of investments)
df_investment2 = pd.DataFrame({'Automotive & Sports Count':d2group['Automotive & Sports Count'].sum(),\
                              'Blanks Count':d2group['Blanks Count'].sum(),\
                              'Cleantech / Semiconductors Count':d2group['Cleantech / Semiconductors Count'].sum(),\
                              'Entertainment Count':d2group['Entertainment Count'].sum(),\
                              'Health Count':d2group['Health Count'].sum(),\
                              'Manufacturing Count':d2group['Manufacturing Count'].sum(),\
                              'News, Search and Messaging Count':d2group['News, Search and Messaging Count'].sum(),\
                              'Others Count':d2group['Others Count'].sum(),\
                              'Social, Finance, Analytics, Advertising Count':d2group['Social, Finance, Analytics, Advertising Count'].sum()},index=[0])

print(pd.DataFrame(df_investment2.max().sort_values(ascending=False)))

# For the top sector count-wise (point 3), which company received the highest investment
d2company1 = D2[D2['Others'] == 1]
d2c1 = d2company1.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(d2c1)

# For the second-best sector count-wise (point 4), which company received the highest investment
d2company2 = D2[D2['Cleantech / Semiconductors'] == 1]
d2c2 = d2company2.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(d2c2)


print("\n\nFunding type = 'venture' and Country='GBR'\n\n")

# Filter data for Country1 and Investment Type between 5 and 15 Mil
D3 = master_frame_mapping[(master_frame_mapping['funding_round_type'] == 'venture') & \
                          (master_frame_mapping['country_code'] == 'IND')]

# total number (or count) of investments for each main sector in a separate column
d3group = (D3.groupby('primary_sector')
        .agg({'raised_amount_usd':'sum',
              'Automotive & Sports':'sum',
              'Blanks':'sum',
              'Cleantech / Semiconductors':'sum',
              'Entertainment':'sum',
              'Health':'sum',
              'Manufacturing':'sum',
              'News, Search and Messaging':'sum', 
              'Others':'sum',
              'Social, Finance, Analytics, Advertising':'sum'})
        .reset_index()
        .rename(columns={'Automotive & Sports':'Automotive & Sports Count',
                         'Blanks':'Blanks Count',
                         'Cleantech / Semiconductors':'Cleantech / Semiconductors Count',
                         'Entertainment':'Entertainment Count',
                         'Health':'Health Count',
                         'Manufacturing':'Manufacturing Count',
                         'News, Search and Messaging':'News, Search and Messaging Count', 
                         'Others':'Others Count',
                         'Social, Finance, Analytics, Advertising':'Social, Finance, Analytics, Advertising Count'}))

d3_total_investment_count = d3group['Automotive & Sports Count'].sum() + \
                            d3group['Blanks Count'].sum() + \
                            d3group['Cleantech / Semiconductors Count'].sum() + \
                            d3group['Entertainment Count'].sum() + \
                            d3group['Health Count'].sum() + \
                            d3group['Manufacturing Count'].sum() + \
                            d3group['News, Search and Messaging Count'].sum() + \
                            d3group['Others Count'].sum() + \
                            d3group['Social, Finance, Analytics, Advertising Count'].sum()

print("Total number of investments (count) : ", '%.0f' % d3_total_investment_count)

#Total amount invested in each main sector in a separate column
d3_total_investment_usd = d3group.raised_amount_usd.sum()
print("Total amount of investment (USD) : ", '%.0f' % d3_total_investment_usd)

# Top sectors (based on count of investments)
df_investment3 = pd.DataFrame({'Automotive & Sports Count':d3group['Automotive & Sports Count'].sum(),\
                              'Blanks Count':d3group['Blanks Count'].sum(),\
                              'Cleantech / Semiconductors Count':d3group['Cleantech / Semiconductors Count'].sum(),\
                              'Entertainment Count':d3group['Entertainment Count'].sum(),\
                              'Health Count':d3group['Health Count'].sum(),\
                              'Manufacturing Count':d3group['Manufacturing Count'].sum(),\
                              'News, Search and Messaging Count':d3group['News, Search and Messaging Count'].sum(),\
                              'Others Count':d3group['Others Count'].sum(),\
                              'Social, Finance, Analytics, Advertising Count':d3group['Social, Finance, Analytics, Advertising Count'].sum()},index=[0])

print(pd.DataFrame(df_investment3.max().sort_values(ascending=False)))

# For the top sector count-wise (point 3), which company received the highest investment
d3company1 = D3[D3['Others'] == 1]
d3c1 = d3company1.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(d3c1)

# For the second-best sector count-wise (point 4), which company received the highest investment
d3company2 = D3[D3['News, Search and Messaging'] == 1]
d3c2 = d3company2.groupby('permalink').agg({'raised_amount_usd':sum}).nlargest(1, 'raised_amount_usd')
print(d3c2)



Table 5.1
----------



Funding type = 'venture' and Country='USA'


Total number of investments (count) :  35759
Total amount of investment (USD) :  422510842796
                                                 0
Others Count                                  8693
Cleantech / Semiconductors Count              8113
Social, Finance, Analytics, Advertising Count 5553
News, Search and Messaging Count              4562
Health Count                                  3403
Manufacturing Count                           2643
Entertainment Count                           1888
Automotive & Sports Count                      566
Blanks Count                                   338
                              raised_amount_usd
permalink                                      
/organization/social-finance         1365199900
                         raised_amount_usd
permalink                                 
/organization/freescale        17600000000


Funding type = 'venture' and Country='GBR'


Total

In [53]:
# Table 6.1 outcome
print("\n\nTable 6.1")
print("----------\n")

plot1df = master_frame[(master_frame['funding_round_type'] == 'venture') | \
                       (master_frame['funding_round_type'] == 'seed')    | \
                       (master_frame['funding_round_type'] == 'angel')   | \
                       (master_frame['funding_round_type'] == 'private_equity')]

plot1df.groupby('funding_round_type').sum().sort_values(by='raised_amount_usd', ascending=False)
plot1df.groupby('funding_round_type').sum().sort_values(by='raised_amount_usd', ascending=False)




Table 6.1
----------



,raised_amount_usd
funding_round_type,
venture,590126216876
private_equity,141925436105
seed,16989864181
angel,4659255123
